In [ ]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run

import network

from block import GENESIS_BLOCK_HASH
from helper import calculate_new_bits
from network import (
    NetworkEnvelope,
    VersionMessage,
)

### Exercise 1

#### Determine what this network message is:

```
f9beb4d976657261636b000000000000000000005df6e0e2
```

In [ ]:
# Exercise 1

message_hex = 'f9beb4d976657261636b000000000000000000005df6e0e2'

### Exercise 2

#### Make [these tests](/edit/code-ch10/network.py) pass.
```
network.py:NetworkTest:test_parse
network.py:NetworkTest:test_serialize
```

In [ ]:
# Exercise 2

reload(network)
run(network.NetworkEnvelopeTest('test_parse'))
run(network.NetworkEnvelopeTest('test_serialize'))

### Exercise 3

#### Make [this test](/edit/code-ch10/network.py) pass.
```
network.py:VersionMessageTest:test_serialize
```

In [ ]:
# Exercise 3

reload(network)
run(network.VersionMessageTest('test_serialize'))

### Exercise 4

#### Make [this test](/edit/code-ch10/network.py) pass.
```
network.py:SimpleNodeTest:test_handshake
```

In [ ]:
# Exercise 4

reload(network)
run(network.SimpleNodeTest('test_handshake'))

### Exercise 5

#### Make [this test](/edit/code-ch10/network.py) pass.
```
network.py:GetHeadersMessageTest:test_serialize
```

In [ ]:
# Exercise 5

reload(network)
run(network.GetHeadersMessageTest('test_serialize'))

In [ ]:
from network import SimpleNode, GetHeadersMessage, HeadersMessage

node = SimpleNode('btc.programmingblockchain.com', testnet=False)
node.handshake()
last_block_hash = GENESIS_BLOCK_HASH
first_epoch_block = None
expected_bits = None
count = 1
while True:
    getheaders = GetHeadersMessage(start_block=last_block_hash)
    node.send(getheaders)
    headers_message = node.wait_for(HeadersMessage)
    for block in headers_message.blocks:
        if not block.check_pow():
            raise RuntimeError('bad proof of work at block {}'.format(count))
        if last_block_hash != GENESIS_BLOCK_HASH and block.prev_block != last_block_hash:  # <2>
            raise RuntimeError('discontinuous block at {}'.format(count))
        if expected_bits and block.bits != expected_bits:
            raise RuntimeError('bad bits at block {} {} vs {}'.format(count, block.bits.hex(), expected_bits.hex()))
        if first_epoch_block and count % 2016 == 2015:
            expected_bits = calculate_new_bits(
                expected_bits, block.timestamp - first_epoch_block.timestamp)
            print(expected_bits.hex())
        elif first_epoch_block is None:
            expected_bits = block.bits
        if count % 2016 == 0 or not first_epoch_block:
            first_epoch_block = block
        count += 1
        last_block_hash = block.hash()
    if len(headers_message.blocks) < 2000:
        break